In [1]:

from multiprocessing import Pool
import numpy as np
import pandas as pd
import queue
import os

from Backtest.strategy import Strategy
from Backtest.event import EventType
from Backtest.backtest import Backtest
from Backtest.data import OHLCDataHandler
from Backtest.portfolio import PortfolioHandler
from Backtest.open_json_gz_files import open_json_gz_files
from Backtest.generate_bars import generate_bars


In [230]:
config = {
    "csv_dir": "C:/backtest/Binance",
    "out_dir": "C:/backtest/results/MOMStrategy",
    "title": "MOMStrategy",
    "is_plot": True,
    "save_plot": True,
    "save_tradelog": True,
    "start_date": pd.Timestamp("2018-01-01T00:0:00", freq="60" + "T"),  # str(freq) + "T"
    "end_date": pd.Timestamp("2018-07-01T00:00:00", freq="60" + "T"),
    "equity": 1.0,
    "freq": 60,  # min
    "commission_ratio": 0.002,
    "suggested_quantity": None,     # None or a value
    "max_quantity": None,           # None or a value, Maximum purchase quantity
    "min_quantity": None,           # None or a value, Minimum purchase quantity
    "min_handheld_cash": None,      # None or a value, Minimum handheld funds
    "exchange": "Binance",
    "tickers": ['BTCUSDT', 'CMTBNB', 'CMTBTC', 'CMTETH', 
                'ETHUSDT', 'LTCUSDT', 'VENBNB', #  'EOSUSDT' 'XRPUSDT'
               'VENBTC', 'VENETH']
}

In [231]:
# trading_data = {}
# for ticker in config['tickers']:
#     # trading_data[ticker] = open_gz_files(config['csv_dir'], ticker)
#     trading_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker + '.h5', key=ticker)

ohlc_data = {}
for ticker in config['tickers']:
    # ohlc_data[ticker] = generate_bars(trading_data, ticker, config['freq'])
    ohlc_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker +'_OHLC_60min.h5', key=ticker)



In [232]:
df = {}
for ticker in config['tickers']:
    aa = ohlc_data[ticker].loc[config['start_date']: config['end_date']]
    df[ticker] = aa['close']
df = pd.DataFrame(df)

In [233]:
def fun(x):
    return (x[-1] - x[0]) / x[0]
df_rolling = df.rolling(window = 10, min_periods = 1).apply(fun).dropna()

In [234]:
df_sort = df_rolling.apply(np.argsort,  axis=1)

In [245]:
def zipppp(x):
    return np.array(sorted(zip(x, range(9)), key = lambda x : x[0]))[:,1]
df_sort = df_sort.apply(zipppp, axis = 1)
df_sort.iloc[0, :] = 8

In [253]:
df_MOM = (df_sort < 4).astype('int32')
df_trading = df_MOM.diff(axis = 0).fillna(0).astype('int32')

In [259]:
df_trading

,BTCUSDT,CMTBNB,CMTBTC,CMTETH,ETHUSDT,LTCUSDT,VENBNB,VENBTC,VENETH
2018-01-01 00:00:00,0,0,0,0,0,0,0,0,0
2018-01-01 01:00:00,0,1,0,0,0,0,1,1,1
2018-01-01 02:00:00,0,0,0,0,0,0,0,0,0
2018-01-01 03:00:00,0,-1,1,1,0,0,-1,0,0
2018-01-01 04:00:00,0,0,0,0,0,1,1,-1,-1
2018-01-01 05:00:00,0,1,-1,0,0,0,0,0,0
2018-01-01 06:00:00,1,-1,1,-1,1,-1,0,0,0
2018-01-01 07:00:00,0,0,0,1,-1,0,-1,1,0
2018-01-01 08:00:00,0,1,0,-1,1,0,0,-1,0
2018-01-01 09:00:00,0,0,0,1,-1,0,0,0,0
